In [1]:
from platypus import NSGAII, Problem, Real
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors
import pandas as pd
import random

In [2]:
# Load the autoreload extension
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [3]:
from Mult_Obj_Helathy import Main_Model as Mdl_H 
from Mult_Obj_Tumor import Main_Model as Mdl_T

# Mixture of CAR NK and WT
(1) check for 50 and 50 

In [4]:
Shell_H = Mdl_H()
Shell_T = Mdl_T()

In [5]:
Tumor_No = 10000
for i in [10.0,5.0,2.5,1.25,0.625,0.3125]:
    x0 = i
    NK_No = x0*Tumor_No
    x = [0.6,0.6]
    param_100 = np.array([0.01, 1.3611247151766386, 2.646954531914279, 1.119928427439465, 0.8030447162325958])  
    frac = param_100[-1]
    MFI_limt = [[(0, 100), (0, 300), (0.0, 50.0)], [(0, 40), (0, 45), (0, 200.0)]]
    Shell_H.Healthy(x[0], frac, x[1], NK_No = NK_No, Tc_No = Tumor_No, param=param_100[:-1], NK_cell='MIX', Tumor_cell='Monocyte',limts=MFI_limt)
    MFI_limt = [[(0, 100), (0, 300), (0.0, 50.0)], [(0, 30), (0, 70), (0, 20.0)]]
    Shell_T.Tumor(x[0], frac, x[1], NK_No = NK_No, Tc_No = Tumor_No, param=param_100[:-1], NK_cell='MIX', Tumor_cell='Kasumi1',limts=MFI_limt)
    print('done')

H 68.50356566167146
T 55.17615988116691
done
H 57.50486808969075
T 43.933876839345174
done
H 45.621544163359815
T 32.564469895504466
done
H 33.01709578631963
T 21.5022249542285
done
H 21.483380275248308
T 12.717463631677074
done
H 12.608021425902683
T 6.974195747985701
done


In [8]:
# Define your multi-objective problem here
class MyProblem(Problem):
    def __init__(self,ET_ratio):
        super(MyProblem, self).__init__(1, 2) # #variables, #objectives)
        self.types[:] = [Real(0.001, 1.0)]
        self.Shell_H = Mdl_H()
        self.Shell_T = Mdl_T()
        self.ET_ratio = ET_ratio
        self.directions[:] = [Problem.MINIMIZE, Problem.MAXIMIZE]

    def evaluate(self, solution):
        x = np.array(solution.variables)
        print('x0',x)
        Tumor_No = 10000
        NK_No = self.ET_ratio*Tumor_No
        param_100 = np.array([0.01, 1.3611247151766386, 2.646954531914279, 1.119928427439465, 0.8030447162325958])  
        frac = param_100[-1]
        alpha1 = 0.6
        MFI_limt = [[(0, 100), (0, 300), (0.0, 50.0)], [(0, 40), (0, 45), (0, 200.0)]]
        f1 = self.Shell_H.Healthy(alpha1, frac, x[0], NK_No = NK_No, Tc_No = Tumor_No, param=param_100[:-1], NK_cell='MIX', Tumor_cell='Monocyte',limts=MFI_limt) # Objective 1
        MFI_limt = [[(0, 100), (0, 300), (0.0, 50.0)], [(0, 30), (0, 70), (0, 20.0)]]
        f2 = self.Shell_T.Tumor(alpha1, frac, x[0], NK_No = NK_No, Tc_No = Tumor_No, param=param_100[:-1], NK_cell='MIX', Tumor_cell='Kasumi1',limts=MFI_limt) # Objective 2
        solution.objectives[:] = [f1, f2]

In [9]:
seed_value = 42
np.random.seed(seed_value)
random.seed(seed_value)
for ET_ratio in [1,5,10]:
        problem = MyProblem(ET_ratio)
        algorithm = NSGAII(problem)
        algorithm.run(2)
        Healthy_Lysis = [s.objectives[0] for s in algorithm.result]
        Tumor_Lysis = [s.objectives[1] for s in algorithm.result]
        alpha4 =  [s.variables[0] for s in algorithm.result]
        filename = f'a4_bw_p001_1_Lysis_Kasumi_Healthy_CAR_NK_a4_ET_{ET_ratio}.xlsx'
        df = pd.DataFrame(columns=['alpha4','Tumor_lysis', 'Healthy_Lysis'])
        df['alpha4'] = alpha4
        df['Tumor_lysis'] = Tumor_Lysis
        df['Healthy_Lysis'] = Healthy_Lysis
        df.to_excel(filename)
        print('Done for', ET_ratio)

x0 [0.63978737]
H 29.128357011268847
T 18.31421253630967
x0 [0.02598574]
H 32.24041742652356
T 24.566354919385137
x0 [0.27575429]
H 29.40219788815137
T 19.120950596208843
x0 [0.22398753]
H 29.37330469065269
T 19.437066996729293
x0 [0.73673474]
H 29.123342426112853
T 18.24469056817505
x0 [0.67702279]
H 29.11649461615201
T 18.283964491588776
x0 [0.89228739]
H 29.095680320655816
T 18.16937747807068
x0 [0.08785189]
H 29.885314615785486
T 21.474509021133905
x0 [0.4224999]
H 29.157237860839924
T 18.615903677686283
x0 [0.03076742]
H 31.70520194695645
T 24.19488477307642
x0 [0.21941934]
H 29.42423269919009
T 19.477096996314913
x0 [0.50584993]
H 29.23111571907716
T 18.46705457470278
x0 [0.02750943]
H 32.05241388892023
T 24.443640812984512
x0 [0.19963881]
H 29.511658110157523
T 19.632018528553306
x0 [0.65023455]
H 29.093121579052294
T 18.298383328281353
x0 [0.54539654]
H 29.131065916355848
T 18.411692399794976
x0 [0.22122018]
H 29.344212149055693
T 19.462607063782535
x0 [0.58967642]
H 29.1016751